**On my honor, as a student, I have neither given nor received unauthorized aid on this academic work.**

In [ ]:
!pip install gensim

In [1]:
# Below is the version of Gensim (Python package for Topic Modeling) I am using in this example
import gensim
print gensim.__version__

3.0.0


In [1]:
import pandas as pd
import csv
from gensim.corpora import Dictionary
from gensim.models import ldamodel
import numpy
%matplotlib inline

import warnings
warnings.filterwarnings('ignore')  # To ignore all warnings that arise here to enhance clarity
# import packages for text processing
import nltk
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
import re

In [3]:
df = pd.read_csv('data/fake.csv')
df.head()

,uuid,ord_in_thread,author,published,title,text,language,crawled,site_url,country,domain_rank,thread_title,spam_score,main_img_url,replies_count,participants_count,likes,comments,shares,type
0,6a175f46bcd24d39b3e962ad0f29936721db70db,0,Barracuda Brigade,2016-10-26T21:41:00.000+03:00,Muslims BUSTED: They Stole Millions In Gov’t B...,Print They should pay all the back all the mon...,english,2016-10-27T01:49:27.168+03:00,100percentfedup.com,US,25689.0,Muslims BUSTED: They Stole Millions In Gov’t B...,0.000,http://bb4sp.com/wp-content/uploads/2016/10/Fu...,0,1,0,0,0,bias
1,2bdc29d12605ef9cf3f09f9875040a7113be5d5b,0,reasoning with facts,2016-10-29T08:47:11.259+03:00,Re: Why Did Attorney General Loretta Lynch Ple...,Why Did Attorney General Loretta Lynch Plead T...,english,2016-10-29T08:47:11.259+03:00,100percentfedup.com,US,25689.0,Re: Why Did Attorney General Loretta Lynch Ple...,0.000,http://bb4sp.com/wp-content/uploads/2016/10/Fu...,0,1,0,0,0,bias
2,c70e149fdd53de5e61c29281100b9de0ed268bc3,0,Barracuda Brigade,2016-10-31T01:41:49.479+02:00,BREAKING: Weiner Cooperating With FBI On Hilla...,Red State : \nFox News Sunday reported this mo...,english,2016-10-31T01:41:49.479+02:00,100percentfedup.com,US,25689.0,BREAKING: Weiner Cooperating With FBI On Hilla...,0.000,http://bb4sp.com/wp-content/uploads/2016/10/Fu...,0,1,0,0,0,bias
3,7cf7c15731ac2a116dd7f629bd57ea468ed70284,0,Fed Up,2016-11-01T05:22:00.000+02:00,PIN DROP SPEECH BY FATHER OF DAUGHTER Kidnappe...,Email Kayla Mueller was a prisoner and torture...,english,2016-11-01T15:46:26.304+02:00,100percentfedup.com,US,25689.0,PIN DROP SPEECH BY FATHER OF DAUGHTER Kidnappe...,0.068,http://100percentfedup.com/wp-content/uploads/...,0,0,0,0,0,bias
4,0206b54719c7e241ffe0ad4315b808290dbe6c0f,0,Fed Up,2016-11-01T21:56:00.000+02:00,FANTASTIC! TRUMP'S 7 POINT PLAN To Reform Heal...,Email HEALTHCARE REFORM TO MAKE AMERICA GREAT ...,english,2016-11-01T23:59:42.266+02:00,100percentfedup.com,US,25689.0,FANTASTIC! TRUMP'S 7 POINT PLAN To Reform Heal...,0.865,http://100percentfedup.com/wp-content/uploads/...,0,0,0,0,0,bias


In [4]:
texts = df['text']
texts = [ i for i in texts ]


In [5]:
df = pd.DataFrame(texts)
df = df.rename(columns={0: 'texts'})
df.head()

,texts
0,Print They should pay all the back all the mon...
1,Why Did Attorney General Loretta Lynch Plead T...
2,Red State : \nFox News Sunday reported this mo...
3,Email Kayla Mueller was a prisoner and torture...
4,Email HEALTHCARE REFORM TO MAKE AMERICA GREAT ...


In [6]:
df['texts'] = df['texts'].str.replace('\d+', ' ')
df['texts'] = df['texts'].str.replace('[^a-zA-Z]+', ' ')
df['texts'] = df['texts'].str.findall('\w{3,}').str.join(' ')
df.to_csv("finaltexts.csv", encoding="utf-8")

In [7]:
df.head()

,texts
0,Print They should pay all the back all the mon...
1,Why Did Attorney General Loretta Lynch Plead T...
2,Red State Fox News Sunday reported this mornin...
3,Email Kayla Mueller was prisoner and tortured ...
4,Email HEALTHCARE REFORM MAKE AMERICA GREAT AGA...


In [9]:
openfile = open('data/finaltexts1.csv', 'rb')

texts = csv.reader(openfile)
texts = [x for y in texts for x in y]


In [10]:
import nltk
nltk.download('stopwords')
nltk.download('wordnet')


[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/linlyn/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /Users/linlyn/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [11]:
ti = texts

In [12]:
# tokenize
texts = [[word for word in text.lower().split() ] for text in texts]

In [13]:
lmtzr = WordNetLemmatizer()
texts = [[lmtzr.lemmatize(word) for word in text ] for text in texts]

In [14]:
stoplist = stopwords.words('english')
texts = [[word for word in text if word not in stoplist] for text in texts]

In [15]:
#remove short words
texts = [[ word for word in tokens if len(word) >= 3 ] for tokens in texts]


In [16]:
# A list of extra stopwords specific to the debates transcripts (if you want to remove more stopwords)
extra_stopwords = ['will', 'people', 'need', 'think', 'well','going', 'can', 'country', 'know', 'lot', 'get','make','way','president', 'want',
                'like','say','got','said','just','something','tell','put','now', 'bad','back','want','right','every','one','use','come','never', 
                'many','along','things','day','also','first','guy', 'great', 'take', 'good', 'much','anderson', 'let', 'would', 'year', 'thing', 'america',
                'talk', 'talking', 'thank', 'does', 'give', 'look', 'believe', 'tonight','today','see','that', 'this', 'em', 'wa', 'http', 'com', 'que','one',
                  'want', 'would', 'also']

extra_stoplist = extra_stopwords
texts = [[word for word in text if word not in extra_stoplist] for text in texts]


In [17]:
# this is text processing required for topic modeling with Gensim
## Create a dictionary representation of the documents.
dictionary = Dictionary(texts)

## Remove rare and common tokens.
# ignore words that appear in less than 5 documents or more than 50% documents (remove too frequent & infrequent words) - an optional step
dictionary.filter_extremes(no_below=5, no_above=0.5) 
# convert words to vetors or integers
corpus = [dictionary.doc2bow(text) for text in texts]

In [18]:
print('Number of unique tokens: %d' % len(dictionary))
print('Number of documents: %d' % len(corpus))

Number of unique tokens: 29705
Number of documents: 12776


# LDA Model Building

In [ ]:
from gensim.models.coherencemodel import CoherenceModel
from gensim.models.wrappers import LdaVowpalWabbit, LdaMallet

In [20]:
%%capture output
numpy.random.seed(1) # setting random seed to get the same results each time. 
model = ldamodel.LdaModel(corpus, id2word=dictionary, num_topics=35, passes=10)

In [21]:
output.show()

In [22]:
%%capture output
# print words without probability
for i in range(0,35):
    topics = model.show_topic(i, 10)
    print ','.join([str(word[0]) for word in topics])

In [23]:
output.show()

israel,der,die,jewish,palestinian,jew,israeli,und,den,von
text,comment,result,please,name,link,post,email,strong,block
area,india,water,time,new,plant,marijuana,technology,chinese,two
time,light,life,around,new,man,men,film,eddie,old
state,government,american,job,tax,percent,obamacare,new,economy,work
russia,russian,war,putin,state,nuclear,military,nato,china,world
change,energy,climate,global,solar,power,lukewearechange,earth,could,gas
election,vote,voter,state,voting,poll,republican,party,candidate,democrat
church,god,christian,south,jesus,north,africa,catholic,ancient,christ
air,aircraft,plane,force,general,ship,navy,retired,defense,military
government,power,world,political,state,war,even,american,nation,public
www,news,infowars,org,video,content,utm,force,twitter,brain
child,woman,school,family,student,life,old,parent,death,girl
email,medium,podesta,information,news,post,facebook,campaign,source,wikileaks
bank,money,million,company,dollar,financial,billion,business,foundation,fund


** Assign the topics to documents in corpus**

In [24]:
%%capture output
lda_corpus = model[corpus]

results = []
for i in lda_corpus:
    print i
    results.append(i)
print 

In [ ]:
output.show()

In [ ]:
# finding highest value from each row
toptopic = [max(collection, key=lambda x: x[1])[0] for collection in results]
toptopic

In [27]:
toptopic = pd.DataFrame(toptopic)
documents = pd.DataFrame(ti)
documents = documents.rename(columns = {0: 'documents'})
summary = documents.join(toptopic)
summary.head()

,documents,0
0,Print They should pay all the back all the mon...,14
1,Why Did Attorney General Loretta Lynch Plead T...,29
2,Red State Fox News Sunday reported this mornin...,29
3,Email Kayla Mueller was prisoner and tortured ...,34
4,Email HEALTHCARE REFORM MAKE AMERICA GREAT AGA...,4


In [ ]:
!pip install pyldavis

In [1]:
import pyLDAvis.gensim

In [2]:
pyLDAvis.enable_notebook()
pyLDAvis.gensim.prepare(model, corpus, dictionary)

NameError: name 'model' is not defined

***Get_term_topic***
**return the particular word in particular topic**

In [31]:
model.get_term_topics('russia')

[(5, 0.037090503691967168)]

In [32]:
model.get_term_topics('water')

[(23, 0.011302869989055716), (24, 0.01416806982657581)]

In [33]:
model.get_term_topics('government')

[(4, 0.012053911413404162),
 (19, 0.011079433012419811),
 (25, 0.018079282002055757)]

**get_document_topics (Predictive Analytics)**

In [35]:
bow = ['russia','government','trump','water']
bow = model.id2word.doc2bow(bow) # convert to bag of words format first
print bow

[(15446, 1), (16530, 1), (28332, 1), (28382, 1)]


In [36]:
doc_topics, word_topics, phi_values = model.get_document_topics(bow, per_word_topics=True)
word_topics

[(15446, [5]), (16530, [5, 24]), (28332, [24]), (28382, [34])]

In [37]:
phi_values

[(15446, [(5, 0.99999999999999989)]),
 (16530, [(5, 0.85819831705572613), (24, 0.14022281257707955)]),
 (28332, [(24, 0.99999999999999911)]),
 (28382, [(34, 0.99999999999999967)])]

In [ ]:
for k, v in dictionary.token2id.iteritems():
    print k, v

In [39]:
unlabeled = ["The Water is one of the energy in the world",
            "hillary clinton email server about america election, and the russia president congraduated to the new president of ammerica",
            "The russia election is not total selected by the russian, it is already sellected",
            "The terrorist is one of the challenges in the world, which will hurt the relationship between countries, and it also challenge human beens' life"]

In [40]:
# Remove useless numbers and alphanumerical words
unlabeled = [re.sub("[^a-zA-Z]+", " ", text) for text in unlabeled]
# tokenize
unlabeled = [[word for word in text.lower().split() ] for text in unlabeled]
# stemming words: having --> have; friends --> friend
lmtzr = WordNetLemmatizer()
unlabeled = [[lmtzr.lemmatize(word) for word in text ] for text in unlabeled]
# remove common words 
stoplist = stopwords.words('english')
unlabeled = [[word for word in text if word not in stoplist] for text in unlabeled]
#remove short words
unlabeled = [[ word for word in tokens if len(word) >= 3 ] for tokens in unlabeled]

In [41]:
for i in unlabeled:
    bow = model.id2word.doc2bow(i)
    doc_topics, word_topics, phi_values = model.get_document_topics(bow, per_word_topics=True)
    print doc_topics

[(6, 0.75714285714285678)]
[(5, 0.13941873499858237), (29, 0.74272412214427441)]
[(5, 0.35673361995955233), (7, 0.50857250248942731)]
[(4, 0.13958111730626532), (16, 0.63977724896271437), (32, 0.11905433214371842)]


# 9. Other Algorithm

In [15]:
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.tokenize import RegexpTokenizer
from nltk.stem import LancasterStemmer
import sklearn

import numpy as np
from collections import Counter

# feature engineering (words to vectors)
from sklearn.feature_extraction.text import TfidfVectorizer

# classification algorithms (or classifiers)
from sklearn.naive_bayes import MultinomialNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC, LinearSVC

# build a pipeline
from sklearn.pipeline import Pipeline

# for gridsearch ... buiild many models with different parameters (e.g., with/without bi-gram)
from sklearn.grid_search import GridSearchCV

# model evaluation, validation
from sklearn.metrics import classification_report, f1_score, accuracy_score, confusion_matrix
from sklearn.cross_validation import StratifiedKFold, cross_val_score, train_test_split 
from sklearn import metrics
from sklearn.metrics import roc_curve, auc

/anaconda/lib/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/anaconda/lib/python2.7/site-packages/sklearn/grid_search.py:43: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


In [53]:
from gensim.models import lsimodel

In [60]:
lsi = lsimodel.LsiModel(corpus, id2word=dictionary, num_topics=35)
for i in range(0,35):
    topics = lsi.show_topic(i, 10)
    print ','.join([str(word[0]) for word in topics])

clinton,trump,state,hillary,obama,american,time,new,war,government
clinton,trump,hillary,war,email,government,election,world,fbi,donald
trump,clinton,email,donald,fbi,obama,hillary,investigation,white,vote
obama,trump,white,house,new,obamacare,donald,administration,war,time
russia,war,syria,russian,military,syrian,time,force,nuclear,trump
infowars,force,utm,brain,retired,general,life,www,content,admiral
retired,general,infowars,utm,admiral,navy,air,brain,rear,major
del,los,por,con,para,gold,una,bank,como,hija
gold,del,state,bank,money,government,debt,los,por,syria
state,russia,gold,russian,election,law,vote,american,government,world
fbi,clinton,election,russia,russian,comey,investigation,email,war,hillary
election,government,russia,trump,vote,syria,fbi,syrian,voter,party
state,election,war,russia,syria,party,white,united,trump,gold
fbi,world,syria,comey,war,email,syrian,american,water,jewish
gold,jewish,black,american,israel,state,police,jew,russia,arab
gold,black,jewish,jew,government

In [61]:
import numpy as np
from sklearn.decomposition import NMF

In [22]:
from sklearn.decomposition import NMF
from sklearn import decomposition
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import NMF, LatentDirichletAllocation
from sklearn.datasets import fetch_20newsgroups

n_topics = 35
n_features = 20
n_top_words = 10

texts = [str(i) for i in texts]
tfidf_vectorizer = TfidfVectorizer(max_df=0.95, min_df=2, stop_words='english')
tfidf = tfidf_vectorizer.fit_transform(texts)
tf_vectorizer = CountVectorizer(max_df=0.95, min_df=2, max_features=n_features,stop_words='english')
tf = tf_vectorizer.fit_transform(texts)



nmf = NMF(n_components=n_topics, random_state=1, alpha=.1, l1_ratio=.5).fit(tfidf)



In [25]:
from __future__ import print_function
def print_top_words(model, feature_names, n_top_words):
    for topic_idx, topic in enumerate(model.components_):
        print("Topic #%d:" % topic_idx)
        print(" ".join([feature_names[i]
                        for i in topic.argsort()[:-n_top_words - 1:-1]]))
    print()
tfidf_feature_names = tfidf_vectorizer.get_feature_names()
print_top_words(nmf, tfidf_feature_names, n_top_words)


Topic #0:
people ha world time wa government like year american right
Topic #1:
weiner abedin email huma anthony fbi laptop investigation device clinton
Topic #2:
trump donald republican president campaign supporter election ha win american
Topic #3:
russia russian putin nato moscow vladimir ukraine military said relation
Topic #4:
text result italic block automatically bold comment strong formating blockquote
Topic #5:
fbi comey investigation email director letter james clinton election server
Topic #6:
pravda fotodom reuters bild afp xiii national kremlin whatsapp volkswagen
Topic #7:
bundy federal ammon jury oregon refuge defendant standoff verdict malheur
Topic #8:
election vote voting voter ballot machine fraud state electoral republican
Topic #9:
http com www youtube utm infowars twitter wearechange watch gmt
Topic #10:
que los del por la para una como est sus
Topic #11:
pipeline dakota water rock standing protester police protector sioux north
Topic #12:
gold market silver price